# Clustering Project - German U-Boats of World War II

In [93]:
import numpy as np
import pandas as pd
import sklearn.preprocessing, sklearn.cluster, sklearn.metrics
import scipy.spatial
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.spatial import distance_matrix
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering, DBSCAN
import matplotlib.pyplot as plt
import seaborn as sns
import requests, zipfile, io
from sklearn.cluster import DBSCAN
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import OPTICS
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score, make_scorer
from plotly.subplots import make_subplots
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import GridSearchCV
from scipy.stats import pearsonr, spearmanr

##### 
- Zdroj dat: https://www.kaggle.com/datasets/cormac42/ww2-u-boats
- Dataset obsahuje informace o německých ponorkách z druhé světové války.
- Data jsou scrapována z wikipedie.


## Preprocessing

In [94]:
# Load the data
df = pd.read_csv('uboats.csv')
df

,Name,Year,Type,Notable Commanders,Warships_sunk_n_total_loss_No,Warships_sunk_n_total_loss_Tons-n-GRT,Warships_Damaged_No,Warships_Damaged_Tons-n-GRT,Merchant_Ships_sunk_No,Merchant_Ships_sunk_GRT,...,Notes,URL,Commissioned,Patrols,Patrols_Count,Wolfpacks,Wolfpacks_Count,Flotilla,Flotilla_Count,Last_Flotilla
0,U-1,1935,IIA,Klaus Ewerth,0,0,0,0,0,0,...,Struck a mine,https://en.wikipedia.org/wiki/German_submarine...,1935-06-29,2 patrols:1st patrol:15 – 29 March 19402nd pat...,2,NaN,0,"{'U-boat School Flotilla': ['1 July 1935 ', ' ...",1,U-boat School Flotilla
1,U-2,1935,IIA,"Hans Heidtmann,Heinrich Liebe,Helmut Rosenbaum...",0,0,0,0,0,0,...,Training boat,https://en.wikipedia.org/wiki/German_submarine...,1935-07-25,2 patrols:1st patrol:15 – 29 March 19402nd pat...,2,NaN,0,"{'U-boat School Flotilla': ['1 July 1935 ', ' ...",2,21st U-boat Flotilla
2,U-3,1935,IIA,"Joachim Schepke,Otto von Bülow,Hans-Hartwig Tr...",0,0,0,0,2,"2,348",...,NaN,https://en.wikipedia.org/wiki/German_submarine...,1935-09-06,5 patrols:1st patrol:4 – 8 September 19392nd p...,5,NaN,0,"{'U-boat School Flotilla': ['1 August 1935 ', ...",2,21st U-boat Flotilla
3,U-4,1935,IIA,Heinz-Otto Schultze,1,"1,090",0,0,3,"5,133",...,NaN,https://en.wikipedia.org/wiki/German_submarine...,1935-08-17,4 patrols:1st patrol:4 – 14 September 19392nd ...,4,NaN,0,"{'U-boat School Flotilla': ['1 August 1935 ', ...",2,21st U-boat Flotilla
4,U-5,1935,IIA,Heinrich Lehmann-Willenbrock,0,0,0,0,0,0,...,Accident,https://en.wikipedia.org/wiki/German_submarine...,1935-08-31,2 patrols:1st patrol:24 August – 8 September 1...,2,NaN,0,{'U-boat School Flotilla': ['1 September 1935 ...,2,21st U-boat Flotilla
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1148,U-4707,1945,XXIII,NaN,0,0,0,0,0,0,...,NaN,https://en.wikipedia.org/wiki/German_submarine...,1945-02-20,NaN,0,NaN,0,"{'5th U-boat Flotilla': ['20 February ', ' 5 M...",1,5th U-boat Flotilla
1149,U-4709,1945,XXIII,NaN,0,0,0,0,0,0,...,NaN,https://en.wikipedia.org/wiki/German_submarine...,1945-03-03,NaN,0,NaN,0,"{'5th U-boat Flotilla': ['3 March ', ' 4 May 1...",1,5th U-boat Flotilla
1150,U-4710,1945,XXIII,Ludwig von Friedeburg,0,0,0,0,0,0,...,NaN,https://en.wikipedia.org/wiki/German_submarine...,1945-05-01,NaN,0,NaN,0,"{'5th U-boat Flotilla': ['1 ', ' 5 May 1945']}",1,5th U-boat Flotilla
1151,U-4711,1945,XXIII,NaN,0,0,0,0,0,0,...,NaN,https://en.wikipedia.org/wiki/German_submarine...,1945-03-21,NaN,0,NaN,0,"{'5th U-boat Flotilla': ['21 March ', ' 4 May ...",1,5th U-boat Flotilla


### Popis datasetu
- Dataset obsahuje 1153 ponerek vyrobených Německem od roku 1935 do roku 1945.
- Neobsahuje ponorky, které byly německem ukořistěny a použity německým námořnictvem.

### Popis sloupců
- **Name** - název ponorky
- **Year** - rok, kdy byla ponorka spuštěna na vodu
- **Type** - typ ponorky
- **Notable Commanders** - významní velitelé ponorky
- **Warships_sunk_n_total_loss_No** - počet potopených válečných lodí
- **Warships_sunk_n_total_loss_Tons-n-GRT** - celková tonáž potopených válečných lodí
- **Warships_Damaged_No** - počet válečných lodí, které byly poškozeny
- **Warships_Damaged_Tons-n-GRT** - celková tonáž válečných lodí, které byly poškozeny
- **Merchant_Ships_sunk_No** - počet potopených obchodních lodí
- **Merchant_Ships_sunk_GRT** - celková tonáž potopených obchodních lodí
- **Merchant_Ships_Damaged_No** - počet obchodních lodí, které byly poškozeny
- **Merchant_Ships_Damaged_GRT** - celková tonáž obchodních lodí, které byly poškozeny
- **Merchant_Ships_total_loss_No** - počet obchodních lodí, které byly ztraceny
- **Merchant_Ships_total_loss_GRT** - celková tonáž obchodních lodí, které byly ztraceny
- **Fate_Event** - osud ponorky
- **Fate_Date** - datum, kdy k osudu ponorky došlo
- **Notes** - poznámky(Např. název lodě, která ponorku potopila; způsob potopení; způsob použití; atd.)
- **URL** - odkaz na wikipedii
- **Commissioned** - datum, kdy byla ponorka zařazena do služby
- **Patrols** - informace o hlídkách vykonaných ponorkou ve formátu: 'Počet hlídek:číslo hlídky:datum začátku - datum konce číslo hlídky:datum začátku - datum konce ...'
- **Patrols_Count** - počet hlídek
- **Wolfpacks** - slovník s informacemi o vlčích smečkách, ve kterých ponorka operovala ve formátu: {'Jméno vlčí smečky': 'datum začátku - datum konce'}
- **Wolfpacks_Count** - počet vlčích smeček
- **Flotilla** - flotily, ke kterým ponorka patřila
- **Flotilla_Count** - počet flotil ke kterým ponorka patřila
- **Last_Flotilla** - poslední flotila, ke které ponorka patřila

In [95]:
# print table where will be each column with its name, type, number of missing values and unique values
def print_columns_descrbition(df):
    tab = pd.DataFrame(columns=['column_name', 'type', 'missing', 'unique'])
    for col in df.columns:
        new_row = pd.DataFrame({'column_name': [col], 'type': [df[col].dtype], 'missing': [df[col].isnull().sum()], 'unique': [df[col].nunique()]})
        tab = pd.concat([tab, new_row], ignore_index=True)
    return tab

# print number of rows and columns
print(f"Table with some description of columns:\nOriginal dataset has {df.shape[0]} rows and {df.shape[1]} columns")
print_columns_descrbition(df)

Table with some description of columns:
Original dataset has 1153 rows and 26 columns


,column_name,type,missing,unique
0,Name,object,0,1153
1,Year,int64,0,11
2,Type,object,0,24
3,Notable Commanders,object,935,169
4,Warships_sunk_n_total_loss_No,int64,0,7
5,Warships_sunk_n_total_loss_Tons-n-GRT,object,0,156
6,Warships_Damaged_No,int64,0,4
7,Warships_Damaged_Tons-n-GRT,object,0,46
8,Merchant_Ships_sunk_No,int64,0,39
9,Merchant_Ships_sunk_GRT,object,0,423


#### Encoding číselných sloupců

In [96]:
# convert columns with numbers which were read as strings to numbers
df['Warships_sunk_n_total_loss_Tons-n-GRT'] = df['Warships_sunk_n_total_loss_Tons-n-GRT'].str.replace(',', '').astype(float)
df['Warships_Damaged_Tons-n-GRT'] = df['Warships_Damaged_Tons-n-GRT'].str.replace(',', '').astype(float)
df['Merchant_Ships_sunk_GRT'] = df['Merchant_Ships_sunk_GRT'].str.replace(',', '').astype(float)
df['Merchant_Ships_damaged_GRT'] = df['Merchant_Ships_damaged_GRT'].str.replace(',', '').astype(float)
df['Merchant_Ships_total_loss_GRT'] = df['Merchant_Ships_total_loss_GRT'].str.replace(',', '').astype(float)

In [97]:
df.describe()

,Year,Warships_sunk_n_total_loss_No,Warships_sunk_n_total_loss_Tons-n-GRT,Warships_Damaged_No,Warships_Damaged_Tons-n-GRT,Merchant_Ships_sunk_No,Merchant_Ships_sunk_GRT,Merchant_Ships_damaged_No,Merchant_Ships_damaged_GRT,Merchant_Ships_total_loss_No,Merchant_Ships_total_loss_GRT,Patrols_Count,Wolfpacks_Count,Flotilla_Count
count,1153.000000,1153.000000,1153.000000,1153.000000,1153.000000,1153.000000,1153.000000,1153.000000,1153.000000,1153.000000,1153.000000,1153.000000,1153.000000,1153.000000
mean,1942.150911,0.263660,622.280139,0.050304,275.049436,2.319167,11554.209020,0.300954,2105.466609,0.061578,387.900260,3.015611,1.754553,2.121422
std,1.846483,0.698366,3146.665601,0.255295,2235.575175,5.824886,30600.823267,0.834670,6127.362156,0.298474,1972.007084,3.604313,3.431312,0.926557
min,1935.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,1941.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
50%,1942.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,2.000000
75%,1943.000000,0.000000,0.000000,0.000000,0.000000,2.000000,8133.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2.000000,3.000000
max,1945.000000,10.000000,46440.000000,3.000000,39346.000000,53.000000,299477.000000,8.000000,62751.000000,3.000000,24338.000000,20.000000,21.000000,7.000000


#### Missing values
- Notable Commanders - ponorky nemusí mít významné velitele
- Wolfpacks - ponorky nemusí být součástí vlčí smečky
- Patrols - ponorky nemusí mít žádné vykonané hlídky
- Notes - poznámky nejsou povinné
- Fate_Date
    - Jedná se o ponorky:
        - Všechny 3 úmyslně zničeny během operace Regenbogen (Datum nejspíš kolem 2-3. května 1945)
        - U-2505 - Doplněno z textu - 3. května 1945
        - U-3004 - Doplněno z textu - 2. května 1945
        - U-3506 - Doplněno z textu - 2. května 1945
- Commissioned
    - Jedná se o ponorky 
        - U-118 - Na wikipedii jsou uvedené 2 datumy, 23. září 1941 a 6. Prosince 1941
                - Bylo vybráno datum 6 Prosince 1941, protože je z důvěryhodnějšího zdroje
        - U-128 - Nejspíš chyba parseru, nejspíš protože na wikipedii je za datumem uvedeno, kým byla ponorka uváděna na vodu
                - opraveno na 12. května 1941

#### Chybné/nevhodné hodnoty
- U-92 - datum osudu může mít několik datumů
    - vybrán 12. listopad 1944 - datum vyřazení z provozu
- U-53 
    - datum osudu je 23 / 24 února 1940
    - bude vybráno 23. února 1940
- U-195, U-219, U-1197, U-1232
    - datum osudu je bez uvedeného dne
        - bude doplněn 15. den(prostřední den měsíce)
- U-984
    - u data osudu je napsáno, že se jedná o přibližnou hodnotu
        - odstranění komentáře
- U některých ponorek je uvedeno více osudných událostí, byla vybrána první
    - U-239 - 24. července 1944, 
    - U-952 - 12. července 1944, vyřazena z provozu
    - U-998 - 27. června 1944, vyřazena z provozu
    - U-1000 - 29. září 1944, vyřazena z provozu
- U-228 
    - u data osudu uveden rok 1944 - 1945
    - opraveno na  4. října 1944, ponorka byla potopena
- U-3501
    - u data osudu není jistý den
    - vybrán jeden z možných dnů, 5. května 1945

In [98]:
# show rows with missinf commission date
df[df['Commissioned'].isnull()]

,Name,Year,Type,Notable Commanders,Warships_sunk_n_total_loss_No,Warships_sunk_n_total_loss_Tons-n-GRT,Warships_Damaged_No,Warships_Damaged_Tons-n-GRT,Merchant_Ships_sunk_No,Merchant_Ships_sunk_GRT,...,Notes,URL,Commissioned,Patrols,Patrols_Count,Wolfpacks,Wolfpacks_Count,Flotilla,Flotilla_Count,Last_Flotilla
113,U-118,1941,XB,NaN,1,925.0,0,0.0,3,14064.0,...,US Avenger aircraft,https://en.wikipedia.org/wiki/German_submarine...,NaN,4 patrols:1st patrol:19 September - 16 October...,4,"{'Wotan': ['5', '7 October 1942'], 'Westwall':...",3,{'4th U-boat Flotilla': ['6 December 1941 - 30...,3,12th U-boat Flotilla
123,U-128,1941,IXC,Ulrich Heyse,0,0.0,0,0.0,12,83639.0,...,USS Moffett & USS Jouett,https://en.wikipedia.org/wiki/German_submarine...,NaN,6 patrols:1st patrol11 – 24 December 19412nd p...,6,NaN,0,"{'2nd U-boat Flotilla': ['12 May 1941 ', ' 17 ...",1,2nd U-boat Flotilla


In [99]:
df.loc[df['Name'] == 'U-118', 'Commissioned'] = pd.to_datetime('1941-12-06')
df.loc[df['Name'] == 'U-128', 'Commissioned'] = pd.to_datetime('1941-05-12')

df[df['Commissioned'].isnull()]

,Name,Year,Type,Notable Commanders,Warships_sunk_n_total_loss_No,Warships_sunk_n_total_loss_Tons-n-GRT,Warships_Damaged_No,Warships_Damaged_Tons-n-GRT,Merchant_Ships_sunk_No,Merchant_Ships_sunk_GRT,...,Notes,URL,Commissioned,Patrols,Patrols_Count,Wolfpacks,Wolfpacks_Count,Flotilla,Flotilla_Count,Last_Flotilla


In [100]:
# show rows with missing fate date
df[df['Fate_Date'].isnull()]

,Name,Year,Type,Notable Commanders,Warships_sunk_n_total_loss_No,Warships_sunk_n_total_loss_Tons-n-GRT,Warships_Damaged_No,Warships_Damaged_Tons-n-GRT,Merchant_Ships_sunk_No,Merchant_Ships_sunk_GRT,...,Notes,URL,Commissioned,Patrols,Patrols_Count,Wolfpacks,Wolfpacks_Count,Flotilla,Flotilla_Count,Last_Flotilla
1028,U-2505,1944,XXI,NaN,0,0.0,0,0.0,0,0.0,...,Elbe II bunker,https://en.wikipedia.org/wiki/German_submarine...,1944-11-07,NaN,0,NaN,0,"{'31st U-boat Flotilla': ['7 November 1944 ', ...",1,31st U-boat Flotilla
1074,U-3004,1944,XXI,NaN,0,0.0,0,0.0,0,0.0,...,Elbe II bunker,https://en.wikipedia.org/wiki/German_submarine...,1944-08-30,NaN,0,NaN,0,"{'4th U-boat Flotilla': ['30 August 1944 ', ' ...",1,4th U-boat Flotilla
1117,U-3506,1944,XXI,Gerhard Thäter,0,0.0,0,0.0,0,0.0,...,Elbe II bunker,https://en.wikipedia.org/wiki/German_submarine...,1944-10-16,NaN,0,NaN,0,"{'8th U-boat Flotilla': ['16 October 1944 ', '...",2,5th U-boat Flotilla


In [101]:
# fill missing fate date as described above
df.loc[df['Name'] == 'U-2505', 'Fate_Date'] = '3 May 1945'
df.loc[df['Name'] == 'U-3004', 'Fate_Date'] = '2 May 1945'
df.loc[df['Name'] == 'U-3506', 'Fate_Date'] = '2 May 1945'

df[df['Fate_Date'].isnull()]

,Name,Year,Type,Notable Commanders,Warships_sunk_n_total_loss_No,Warships_sunk_n_total_loss_Tons-n-GRT,Warships_Damaged_No,Warships_Damaged_Tons-n-GRT,Merchant_Ships_sunk_No,Merchant_Ships_sunk_GRT,...,Notes,URL,Commissioned,Patrols,Patrols_Count,Wolfpacks,Wolfpacks_Count,Flotilla,Flotilla_Count,Last_Flotilla


In [102]:
# correct fate date for U-92 as described above
df.loc[df['Name'] == 'U-92', 'Fate_Date'] = '12 November 1944'
df.loc[df['Name'] == 'U-92', 'Fate_Event'] = 'Decommissioned'

In [103]:
# replace 23 / 24 February 1940 with 23 February 1940 on U-53 as described above
df.loc[df['Name'] == 'U-53', 'Fate_Date'] = '23 February 1940'

df[df['Name'] == 'U-195']

,Name,Year,Type,Notable Commanders,Warships_sunk_n_total_loss_No,Warships_sunk_n_total_loss_Tons-n-GRT,Warships_Damaged_No,Warships_Damaged_Tons-n-GRT,Merchant_Ships_sunk_No,Merchant_Ships_sunk_GRT,...,Notes,URL,Commissioned,Patrols,Patrols_Count,Wolfpacks,Wolfpacks_Count,Flotilla,Flotilla_Count,Last_Flotilla
190,U-195,1942,IXD1,NaN,0,0.0,0,0.0,2,14391.0,...,Became Japanese sub I-506,https://en.wikipedia.org/wiki/German_submarine...,1942-09-05,3 patrols:1st patrol:20 March – 23 July 19432n...,3,NaN,0,"{'4th U-boat Flotilla': ['5 September 1942 ', ...",3,33rd U-boat Flotilla


In [104]:
# find fate date without specified day - has only 1 space
df[df['Fate_Date'].str.count(' ') == 1]

,Name,Year,Type,Notable Commanders,Warships_sunk_n_total_loss_No,Warships_sunk_n_total_loss_Tons-n-GRT,Warships_Damaged_No,Warships_Damaged_Tons-n-GRT,Merchant_Ships_sunk_No,Merchant_Ships_sunk_GRT,...,Notes,URL,Commissioned,Patrols,Patrols_Count,Wolfpacks,Wolfpacks_Count,Flotilla,Flotilla_Count,Last_Flotilla
190,U-195,1942,IXD1,NaN,0,0.0,0,0.0,2,14391.0,...,Became Japanese sub I-506,https://en.wikipedia.org/wiki/German_submarine...,1942-09-05,3 patrols:1st patrol:20 March – 23 July 19432n...,3,NaN,0,"{'4th U-boat Flotilla': ['5 September 1942 ', ...",3,33rd U-boat Flotilla
214,U-219,1942,XB,NaN,0,0.0,0,0.0,0,0.0,...,Became Japanese sub I-505,https://en.wikipedia.org/wiki/German_submarine...,1942-12-12,2 patrols:1st patrol:22 October 1943 - 1 Janua...,2,NaN,0,{'4th U-boat Flotilla': ['12 December 1942 - 3...,3,33rd U-boat Flotilla
925,U-1197,1943,VIIC,NaN,0,0.0,0,0.0,0,0.0,...,NaN,https://en.wikipedia.org/wiki/German_submarine...,1943-12-02,NaN,0,NaN,0,"{'21st U-boat Flotilla': ['2 December 1943 ', ...",2,31st U-boat Flotilla
950,U-1232,1943,IXC/40,Kurt Dobratz,0,0.0,0,0.0,3,17355.0,...,NaN,https://en.wikipedia.org/wiki/German_submarine...,1944-03-08,1 patrol:10 November 1944 – 14 February 1945,1,NaN,0,"{'31st U-boat Flotilla': ['8 March ', ' 31 Oct...",2,33rd U-boat Flotilla


In [105]:
# for u-boats with fate date without day, set day to 15
df.loc[df['Fate_Date'].str.count(' ') == 1, 'Fate_Date'] ='15 ' + df.loc[df['Fate_Date'].str.count(' ') == 1, 'Fate_Date'] 

df[df['Name'] == 'U-195']

,Name,Year,Type,Notable Commanders,Warships_sunk_n_total_loss_No,Warships_sunk_n_total_loss_Tons-n-GRT,Warships_Damaged_No,Warships_Damaged_Tons-n-GRT,Merchant_Ships_sunk_No,Merchant_Ships_sunk_GRT,...,Notes,URL,Commissioned,Patrols,Patrols_Count,Wolfpacks,Wolfpacks_Count,Flotilla,Flotilla_Count,Last_Flotilla
190,U-195,1942,IXD1,NaN,0,0.0,0,0.0,2,14391.0,...,Became Japanese sub I-506,https://en.wikipedia.org/wiki/German_submarine...,1942-09-05,3 patrols:1st patrol:20 March – 23 July 19432n...,3,NaN,0,"{'4th U-boat Flotilla': ['5 September 1942 ', ...",3,33rd U-boat Flotilla


In [106]:
# remove comment from fate date of U-984
df.loc[df['Name'] == 'U-984', 'Fate_Date'] = '2 August 1944'
df

,Name,Year,Type,Notable Commanders,Warships_sunk_n_total_loss_No,Warships_sunk_n_total_loss_Tons-n-GRT,Warships_Damaged_No,Warships_Damaged_Tons-n-GRT,Merchant_Ships_sunk_No,Merchant_Ships_sunk_GRT,...,Notes,URL,Commissioned,Patrols,Patrols_Count,Wolfpacks,Wolfpacks_Count,Flotilla,Flotilla_Count,Last_Flotilla
0,U-1,1935,IIA,Klaus Ewerth,0,0.0,0,0.0,0,0.0,...,Struck a mine,https://en.wikipedia.org/wiki/German_submarine...,1935-06-29,2 patrols:1st patrol:15 – 29 March 19402nd pat...,2,NaN,0,"{'U-boat School Flotilla': ['1 July 1935 ', ' ...",1,U-boat School Flotilla
1,U-2,1935,IIA,"Hans Heidtmann,Heinrich Liebe,Helmut Rosenbaum...",0,0.0,0,0.0,0,0.0,...,Training boat,https://en.wikipedia.org/wiki/German_submarine...,1935-07-25,2 patrols:1st patrol:15 – 29 March 19402nd pat...,2,NaN,0,"{'U-boat School Flotilla': ['1 July 1935 ', ' ...",2,21st U-boat Flotilla
2,U-3,1935,IIA,"Joachim Schepke,Otto von Bülow,Hans-Hartwig Tr...",0,0.0,0,0.0,2,2348.0,...,NaN,https://en.wikipedia.org/wiki/German_submarine...,1935-09-06,5 patrols:1st patrol:4 – 8 September 19392nd p...,5,NaN,0,"{'U-boat School Flotilla': ['1 August 1935 ', ...",2,21st U-boat Flotilla
3,U-4,1935,IIA,Heinz-Otto Schultze,1,1090.0,0,0.0,3,5133.0,...,NaN,https://en.wikipedia.org/wiki/German_submarine...,1935-08-17,4 patrols:1st patrol:4 – 14 September 19392nd ...,4,NaN,0,"{'U-boat School Flotilla': ['1 August 1935 ', ...",2,21st U-boat Flotilla
4,U-5,1935,IIA,Heinrich Lehmann-Willenbrock,0,0.0,0,0.0,0,0.0,...,Accident,https://en.wikipedia.org/wiki/German_submarine...,1935-08-31,2 patrols:1st patrol:24 August – 8 September 1...,2,NaN,0,{'U-boat School Flotilla': ['1 September 1935 ...,2,21st U-boat Flotilla
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1148,U-4707,1945,XXIII,NaN,0,0.0,0,0.0,0,0.0,...,NaN,https://en.wikipedia.org/wiki/German_submarine...,1945-02-20,NaN,0,NaN,0,"{'5th U-boat Flotilla': ['20 February ', ' 5 M...",1,5th U-boat Flotilla
1149,U-4709,1945,XXIII,NaN,0,0.0,0,0.0,0,0.0,...,NaN,https://en.wikipedia.org/wiki/German_submarine...,1945-03-03,NaN,0,NaN,0,"{'5th U-boat Flotilla': ['3 March ', ' 4 May 1...",1,5th U-boat Flotilla
1150,U-4710,1945,XXIII,Ludwig von Friedeburg,0,0.0,0,0.0,0,0.0,...,NaN,https://en.wikipedia.org/wiki/German_submarine...,1945-05-01,NaN,0,NaN,0,"{'5th U-boat Flotilla': ['1 ', ' 5 May 1945']}",1,5th U-boat Flotilla
1151,U-4711,1945,XXIII,NaN,0,0.0,0,0.0,0,0.0,...,NaN,https://en.wikipedia.org/wiki/German_submarine...,1945-03-21,NaN,0,NaN,0,"{'5th U-boat Flotilla': ['21 March ', ' 4 May ...",1,5th U-boat Flotilla


In [107]:
# show unique fate events
df['Fate_Event'].unique()

array(['Sunk', 'Stricken', 'Scuttled', 'Scrapped', 'Missing',
       'Decommissioned', 'Captured', 'Surrendered', 'Broken up', 'Sold',
       'Interned, then sold', 'Ran aground & destroyed', 'Destroyed',
       'Interned', 'Grounded', 'Buried', 'Aground'], dtype=object)

In [108]:
# show fate date with len < 10
# print the names of u-boats with fate date with length less than 10
df[df['Fate_Date'].str.len() < 10]['Fate_Event']

234    Broken up
809    Broken up
854     Scrapped
856     Scrapped
Name: Fate_Event, dtype: object

In [109]:
df[df['Fate_Date'].str.len() < 10]['Name']

234     U-239
809     U-952
854     U-998
856    U-1000
Name: Name, dtype: object

In [110]:
# correct fate date for U-239, U-952, U-998, U-1000
df.loc[df['Name'] == 'U-239', 'Fate_Date'] = '24 July 1944'
df.loc[df['Name'] == 'U-952', 'Fate_Date'] = '12 July 1944'
df.loc[df['Name'] == 'U-952', 'Fate_Event'] = 'Decommissioned'
df.loc[df['Name'] == 'U-998', 'Fate_Date'] = '27 June 1944'
df.loc[df['Name'] == 'U-998', 'Fate_Event'] = 'Decommissioned'
df.loc[df['Name'] == 'U-1000', 'Fate_Date'] = '29 September 1944'
df.loc[df['Name'] == 'U-1000', 'Fate_Event'] = 'Decommissioned'

In [111]:
# show fate date which contains character '-'
df[df['Fate_Date'].str.contains('–')]

,Name,Year,Type,Notable Commanders,Warships_sunk_n_total_loss_No,Warships_sunk_n_total_loss_Tons-n-GRT,Warships_Damaged_No,Warships_Damaged_Tons-n-GRT,Merchant_Ships_sunk_No,Merchant_Ships_sunk_GRT,...,Notes,URL,Commissioned,Patrols,Patrols_Count,Wolfpacks,Wolfpacks_Count,Flotilla,Flotilla_Count,Last_Flotilla
223,U-228,1942,VIIC,NaN,0,0.0,0,0.0,0,0.0,...,Shot down 2 aircraft,https://en.wikipedia.org/wiki/German_submarine...,1942-09-12,6 patrols:1st patrol:6 February – 29 March 194...,6,NaN,0,"{'5th U-boat Flotilla': ['12 September 1942 ',...",2,6th U-boat Flotilla


In [112]:
# correct fate date and event for U-228
df.loc[df['Name'] == 'U-228', 'Fate_Date'] = '4 October 1944'
df.loc[df['Name'] == 'U-228', 'Fate_Event'] = 'Sunk'

In [113]:
df[df['Fate_Date'].str.contains('or')]

,Name,Year,Type,Notable Commanders,Warships_sunk_n_total_loss_No,Warships_sunk_n_total_loss_Tons-n-GRT,Warships_Damaged_No,Warships_Damaged_Tons-n-GRT,Merchant_Ships_sunk_No,Merchant_Ships_sunk_GRT,...,Notes,URL,Commissioned,Patrols,Patrols_Count,Wolfpacks,Wolfpacks_Count,Flotilla,Flotilla_Count,Last_Flotilla
1112,U-3501,1944,XXI,NaN,0,0.0,0,0.0,0,0.0,...,NaN,https://en.wikipedia.org/wiki/German_submarine...,1944-07-29,NaN,0,NaN,0,"{'8th U-boat Flotilla': ['29 July ', ' 1 Novem...",2,5th U-boat Flotilla


In [114]:
# correct fate date for U-3501
df.loc[df['Name'] == 'U-3501', 'Fate_Date'] = '5 May 1945'

In [115]:
# encode datetime columns
# comissioned to column comissioned_year, comissioned_month, comissioned_day
df['Commissioned'] = pd.to_datetime(df['Commissioned'])
df['Commissioned_year'] = df['Commissioned'].dt.year
df['Commissioned_month'] = df['Commissioned'].dt.month
df['Commissioned_day'] = df['Commissioned'].dt.day

# make a cosine of month and day so that the model can understand the cyclical nature of time
df['Commissioned_month_cos'] = np.cos(2 * np.pi * df['Commissioned_month'] / 12)
df['Commissioned_day_cos'] = np.cos(2 * np.pi * df['Commissioned_day'] / 31)


# fate date to column fate_year, fate_month, fate_day
df['Fate_Date'] = pd.to_datetime(df['Fate_Date'])
df['Fate_year'] = df['Fate_Date'].dt.year
df['Fate_month'] = df['Fate_Date'].dt.month
df['Fate_day'] = df['Fate_Date'].dt.day

# make a cosine of month and day so that the model can understand the cyclical nature of time
df['Fate_month_cos'] = np.cos(2 * np.pi * df['Fate_month'] / 12)
df['Fate_day_cos'] = np.cos(2 * np.pi * df['Fate_day'] / 31)




In [118]:
print_columns_descrbition(df)

,column_name,type,missing,unique
0,Name,object,0,1153
1,Year,int64,0,11
2,Type,object,0,24
3,Notable Commanders,object,935,169
4,Warships_sunk_n_total_loss_No,int64,0,7
5,Warships_sunk_n_total_loss_Tons-n-GRT,float64,0,156
6,Warships_Damaged_No,int64,0,4
7,Warships_Damaged_Tons-n-GRT,float64,0,46
8,Merchant_Ships_sunk_No,int64,0,39
9,Merchant_Ships_sunk_GRT,float64,0,423


In [116]:
# TODO - drop columns which will not be used
# TODO - missing values
# TODO - encoding
# TODO - scaling
# TODO - slocit sunk and destroyeD?
# TODO - correlation
# TODO - outliers
# TODO - popis co jsme zatim zjistili

In [117]:
# TODO - clustering 3 methods (kmeans...)
# - on scaled and unscaled data
# only with numerical columns 